In [1]:
import pandas as pd
import numpy as np
import re

In [28]:
laptop = pd.read_csv("../laptop.csv")
detailed_df = pd.read_csv('detailed_df.csv')
backup = pd.read_csv('../backup.csv')
df2 = detailed_df.copy()

In [30]:
laptop['storage capacity'] = laptop['storage capacity'].apply(lambda x: int(x))
laptop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1540 entries, 0 to 1539
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        1540 non-null   int64  
 1   id                1540 non-null   object 
 2   price             1540 non-null   int64  
 3   brand             1540 non-null   object 
 4   model             1540 non-null   object 
 5   processor         1540 non-null   object 
 6   ram memory        1540 non-null   int64  
 7   display size      1540 non-null   float64
 8   storage capacity  1540 non-null   int64  
 9   cpu cores         1215 non-null   object 
 10  graphics card     0 non-null      float64
 11  graphics memory   240 non-null    object 
 12  rating            1540 non-null   float64
 13  reviews           1540 non-null   int64  
 14  name              1540 non-null   object 
 15  url               1540 non-null   object 
dtypes: float64(3), int64(5), object(8)
memory 

In [24]:
# laptop.loc[laptop['cpu cores'] == 'Quad Core', 'cpu cores'] = '4'
# laptop.loc[laptop['cpu cores'] == 'Octa Core', 'cpu cores'] = '8'
# # laptop.loc[laptop['cpu cores'] == 'Hexa Core', 'cpu cores'] = '6'
# # laptop.loc[laptop['cpu cores'] == 'Single Core', 'cpu cores'] = '1'
# # # laptop.loc[laptop['cpu cores'] == 'Dual Core', 'cpu cores'] = '2'
# laptop.loc[laptop['storage capacity'] == '1TB', 'storage capacity'] = '1024'
# laptop.loc[laptop['storage capacity'] == '4', 'storage capacity'] = '256'



laptop['storage capacity'].value_counts()
# laptop['cpu cores'].isnull().sum()

storage capacity
512     854
256     373
1024    281
128      25
2048      5
64        2
Name: count, dtype: int64

In [19]:
laptop['cpu cores'].isnull().sum()

325

In [15]:
def clean_text(text):
    if isinstance(text, str):  # Check if the text is a string
        # Define the pattern to match unwanted characters
        pattern = r'[\/,.\(\):;\-]'
        # Use re.sub() to replace unwanted characters with an empty string
        cleaned_text = re.sub(pattern, '', text)
        return cleaned_text
    return text  # If the text is not a string, return it as is

In [32]:
df2['details'] = df2['details'].astype(str)
df2['details'] = df2['details'].apply(clean_text)

In [46]:

df2['details'] = df2['details'].str.replace('\n', ' ')
df2['details'] = df2['details'].str.replace(r'(\d+)\s+cores', r'\1cores', regex=True)


# Iterate over the dataframe rows
for index, row in df2.iterrows():
    # Check for null or "Not Specified" in 'details' column
    if pd.isnull(row['details']) or row['details'] == 'Not Specified':
        continue

    # Split the details into parts
    name_parts = row['details'].split(' ')

    # Initialize a variable to store the number of cores
    cores = None

    # Check each part for core counts
    for part in name_parts:
        if part == '2Cores':
            cores = 2
            break
        elif part == '4Cores':
            cores = 4
            break
        elif part == '6Cores':
            cores = 6
            break
        elif part in ['8-core', '8Cores']:
            cores = 8
            break
        elif part == '10Cores':
            cores = 10
            break
        elif part == '12Cores':
            cores = 12
            break
        elif part == '1Core':
            cores = 1
            break

    # Assign the cores to the dataframe if found
    if cores is not None:
        df2.loc[index, 'cpu cores'] = cores
        print("Row:", index)
        print("Cores:", df2.loc[index, 'cpu cores'])
        print("Name:", row['name'])


# detailed_df = df2.copy()

Row: 688
Cores: 6
Name: HP EliteBook 845 G8 Ryzen 5 Pro 5650U/16GB RAM/ 512GB SSD/14" FHD IPS/Backlit Keyboard/ Win 11 Pro Genuine (Business Series)
Row: 792
Cores: 4
Name: HP Laptop 15-dy2795wm /i5 1135G7 / 8gb DDR4 Ram /256gb SSD / 15 inch FHD LED Display
Row: 975
Cores: 2
Name: Lenovo IdeaPad Flex 3 11ADA05 2021 AMD Athlon / 4GB RAM / 256GB SSD / 11.6" FHD 360 Touch display
Row: 1008
Cores: 2
Name: Asus VivoBook X513E 11th Gen Core i3-1115G4 Processor, 4GB RAM, 256GB SSD, 15.6â Full HD (1920x1080) 60Hz Display Windows 11 Genuine Fingerprint,Abbsy Blue
Row: 1033
Cores: 4
Name: Lenovo Slim 7 With i7 10th Gen Processor 14 Inch FHD IPS Display 8GB RAM 256GB ROM
Row: 1490
Cores: 6
Name: Lenovo ThinkBook 13s AMD Ryzen 5 6600U – 8GB DDR5 RAM – 256GB NVMe SSD – 13.3″ FHD Display – Windows 11 Pro – FingerPrint Reader – Backlit Keyboard


In [56]:
# df2['cpu cores'] = df2['cpu cores'].apply(lambda x: int(x) if pd.notna(x) and x != 'Not Specified' else np.nan if x == 'Not Specified' else x)
df2['cpu cores'] = df2['cpu cores'].apply(lambda x: int(x) if pd.notnull(x) and x != 'Not Specified' else x)
df2['cpu cores'].value_counts()

cpu cores
4                315
Not Specified    312
8                208
10               133
2                 95
6                 93
1                 36
12                33
14                 1
0                  1
Name: count, dtype: int64

In [55]:
df2['cpu cores'].isnull().sum()

313

In [14]:
# columns = list(df2.columns)
# columns[0], columns[2] = columns[2], columns[0]
# df2 = df2[columns]

# df2.to_csv('detailed_df.csv', index=False)


# laptop['cpu cores'] = backup['cpu cores']


# laptop.info()

In [31]:
laptop = backup.copy()
laptop.to_csv("../laptop.csv", index=False)
backup.to_csv('../backup.csv')